In [1]:
import pandas as pd
import numpy as np 
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import random

In [2]:
N = 3 # tunable?
random.seed(42)

In [3]:
drug_response_data = pd.read_csv("/data/yingfei/cancer_data/cell_drug_auc_final_1111.csv")
print(drug_response_data.shape)
drug_response_data.head()

(692, 386)


,ARXSPAN_ID,JW-7-24-1,KIN001-260,NSC-87877,GNE-317,NAVITOCLAX,PLX-4720,ERK5-IN-1,VX-11E,TGX-221,...,BIX02189,ISPINESIB MESYLATE,KIN001-135,KIN001-206,KIN001-236,KIN001-266,LUMINESPIB,NUTLIN-3A,SGC0946,SL 0101-1
0,ACH-000001,0.778432,0.951321,0.840287,-99999.000000,-99999.000000,0.936410,0.891908,0.402122,0.659600,...,0.982272,0.733492,0.072277,0.983536,-99999.000000,0.651746,0.982084,0.893777,0.983861,0.933035
1,ACH-000002,0.788327,0.874760,0.760137,0.905754,0.930764,0.991068,0.278288,0.972905,0.558774,...,0.899628,0.604228,0.046061,0.918265,0.865628,-99999.000000,0.985921,0.839175,0.988639,0.533204
2,ACH-000004,0.737630,0.938733,0.835441,0.808964,0.966133,0.983552,-99999.000000,0.993912,0.596027,...,0.965245,0.782295,0.366912,0.844979,0.761323,-99999.000000,0.980945,0.946694,0.992605,0.761956
3,ACH-000006,0.176396,0.571569,0.581729,0.585938,0.958510,0.935574,0.600428,-99999.000000,0.596056,...,0.834158,0.605360,0.124930,0.813384,0.782187,-99999.000000,0.971058,0.871277,-99999.000000,0.495020
4,ACH-000007,0.569751,0.755449,0.954924,0.939587,0.965638,0.932666,-99999.000000,0.956481,0.838692,...,0.927519,0.812955,0.719780,0.815099,0.795736,0.860768,0.983317,0.888067,0.987564,0.787948


In [4]:
cell_line_mutation = pd.read_csv("/data/yingfei/cancer_data/mutations_raw_324_final.csv")
print(cell_line_mutation.shape)
cell_line_mutation.head()

(692, 325)


,DepMap_ID,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,FAM123B,APC,...,VEGFA,VHL,WHSC1,WHSC1L1,WT1,XPO1,XRCC2,ZNF217,ZNF703,TERC
0,ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,ACH-000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,ACH-000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,ACH-000007,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
drug_smiles_data = pd.read_csv("/data/yingfei/cancer_data/drug_smiles.csv", header = None, names = ['drug_name', 'smiles'])
print(drug_smiles_data.shape)
drug_smiles_data.head()

(408, 2)


,drug_name,smiles
0,JW-7-24-1,COC1=CC(=CC(=C1)C2=CC3=C4C(=CN=C3C=C2)C=CC(=O)...
1,KIN001-260,C1CC1COC2=CC=CC(=O)C2=C3C=C(C(=C(N3)N)C#N)C4CC...
2,NSC-87877,C1=CC2=C(C(=O)C(=NNC3=CC4=C(C=C3)C=C(C=C4)S(=O...
3,GNE-317,CC1=C(SC2=C1N=C(N=C2N3CCOCC3)C4=CN=C(N=C4)N)C5...
4,NAVITOCLAX,CC1(CCC(=C(C1)CN2CCN(CC2)C3=CC=C(C=C3)C(=O)NS(...


In [6]:
### drug recommendation task [given the prompt with a cell line and N candidate drugs -> predict the most suitable drug and provide reasoning]

### Input Prompt:
# "Think step by step and decide the best drug option for the cell line with given mutations: [Drug Name], [Reasoning]
# Drug 1: The drug is AGI-6780. The drug SMILES structure is … Drug Target pathway: Metabolism. 
# Drug 2: The drug is JW-7-24-1. The drug SMILES structure is … Drug Target pathway: Other, kinases. 
# Drug 3: The drug is KIN001-260. The drug SMILES structure is …Drug Target pathway: DNA replication. 
# The mutations of the cell lines are NOTCH1 NOTCH3 PIK3R1 PPP2R1A TP53 TSC2 WHSCL1. 
# Best drug option: ?
### Output Groundtruth: ?

### Data needed for prompt: cell line mutation data, drug information (drug name, drug SMILES structure, target pathway [KEGG/database])
### Data needed for output: cell line drug response prediction (drug with lower AUDRC value indicates better response)

### Intermediate dataset: Cell line name, mutation status columns, Drug 1 name, Drug 1 SMILES, Drug 1 target pathway, Drug 1 response, Drug 2 name, ...
### Initiliaze the data
interm_data = {
    'cell_id': [],
    'mut_status': []
}

for i in range(N):
    drug_id = f'drug_{i+1}'
    interm_data[f'{drug_id}_name'] = []
    interm_data[f'{drug_id}_SMILES'] = []
    interm_data[f'{drug_id}_target'] = []
    interm_data[f'{drug_id}_target_path'] = []
    interm_data[f'{drug_id}_response'] = []

In [7]:
### Fill in data features
gene_columns = cell_line_mutation.columns[1:]
interm_data['cell_id'] = cell_line_mutation['DepMap_ID']
interm_data['mut_status'] = cell_line_mutation.apply(lambda row: ', '.join(gene for gene in gene_columns if row[gene] == 1), axis=1)

# top_n_drugs = drug_response_data.drop('ARXSPAN_ID', axis=1).apply(lambda row: ', '.join(row[row > 0].nsmallest(N).index), axis=1)
# top_n_drug_response = drug_response_data.drop('ARXSPAN_ID', axis=1).apply(lambda row: list(row[row > 0].nsmallest(3)), axis=1)

### simple version: three drugs - one from top 5%; one from bottom 5%, another from the IQR?
# three drug candidates
drug_response_subset = drug_response_data[drug_response_data.columns[1:]].copy()

drug_response_subset['threshold_low'] = drug_response_subset[drug_response_subset>0].quantile(0.05, axis = 1)
for i in range(len(drug_response_subset)):
    drug_id = f"drug_1"
    boo_val_lst = (drug_response_subset.iloc[0] < drug_response_subset.loc[0, 'threshold_low']) & (drug_response_subset.iloc[0] > 0)
    true_indices = [index for index, value in enumerate(boo_val_lst) if value]
    selected_index = random.choice(true_indices) if true_indices else None
    interm_data[f'{drug_id}_name'].append(drug_response_subset.columns.values[selected_index])
    interm_data[f'{drug_id}_response'].append(drug_response_subset.iloc[0].values[selected_index])

drug_response_subset['threshold_25'] = drug_response_subset[drug_response_subset>0].quantile(0.25, axis = 1)
drug_response_subset['threshold_75'] = drug_response_subset[drug_response_subset>0].quantile(0.75, axis = 1)
for i in range(len(drug_response_subset)):
    drug_id = f"drug_2"
    boo_val_lst = (drug_response_subset.iloc[0] > drug_response_subset.loc[0, 'threshold_25']) & (drug_response_subset.iloc[0] < drug_response_subset.loc[0, 'threshold_75'])
    true_indices = [index for index, value in enumerate(boo_val_lst) if value]
    selected_index = random.choice(true_indices) if true_indices else None
    interm_data[f'{drug_id}_name'].append(drug_response_subset.columns.values[selected_index])
    interm_data[f'{drug_id}_response'].append(drug_response_subset.iloc[0].values[selected_index])

drug_response_subset['threshold_high'] = drug_response_subset[drug_response_subset>0].quantile(0.95, axis = 1)
for i in range(len(drug_response_subset)):
    drug_id = f"drug_3"
    boo_val_lst = (drug_response_subset.iloc[0] > drug_response_subset.loc[0, 'threshold_high']) & (drug_response_subset.iloc[0] > 0)
    true_indices = [index for index, value in enumerate(boo_val_lst) if value]
    selected_index = random.choice(true_indices) if true_indices else None
    interm_data[f'{drug_id}_name'].append(drug_response_subset.columns.values[selected_index])
    interm_data[f'{drug_id}_response'].append(drug_response_subset.iloc[0].values[selected_index])

In [8]:
def find_drug_page(drug_name):
    url = f'https://www.cancerrxgene.org/search?query={drug_name}'
    response = requests.get(url)
    
    if response.ok:
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            link_tag = soup.table.find_all('a', href=True)
            link = link_tag[0]['href']
            return link
        except Exception as e:
            # print(f"Failed to retrieve the web page for drug {drug_name}, {e}")
            return ""

In [9]:
def drug_info_ws(drug_name):
    try:
        url = f'{find_drug_page(drug_name)}/overview/ic50'
        response = requests.get(url)
    except Exception as e:
        # print(f"Failed to retrieve the web page overview for drug {drug_name}, {e}")
        return {'name': "", 'target': "", 'pathway': ""}

    if response.ok:
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            compound = soup.find('h3', class_='emboldened')
            drug_target = compound.find_next_sibling('h4')
            drug_target_pathway = drug_target.find_next_sibling('h4')
            return {'name': compound, 'target': drug_target, 'pathway': drug_target_pathway}
        except Exception as e:
            # print(f"Failed to retrieve the web page for drug {drug_name}, {e}")
            return {'name': compound, 'target': "", 'pathway': ""}
    else:
        # print(f"Failed to retrieve the web page for drug {drug_name}. Status code: {response.status_code}")
        return {'name': "", 'target': "", 'pathway': ""}

In [10]:
def drug_info_ws_v2(drug_name): ### faster
    try:
        url = f'https://www.cancerrxgene.org/search?query={drug_name}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        drug_info_table = soup.find_all('table')[0].find_all('td')
        drug_target = drug_info_table[2].text.strip()
        drug_target_pathway = drug_info_table[3].text.strip()
        return {'name': drug_name, 'target': drug_target, 'pathway': drug_target_pathway}
    except Exception as e:
        # print(f"Failed to retrieve the web page overview for drug {drug_name}, {e}")
        return {'name': "", 'target': "", 'pathway': ""}

In [11]:
### Drug features
not_avail_drug_lst = []
for n in range(N):
    drug_id = f"drug_{n+1}"
    for i in tqdm(range(len(interm_data['cell_id']))):
        name = interm_data[f'{drug_id}_name'][i]
        smiles = drug_smiles_data.loc[drug_smiles_data.drug_name == name, 'smiles'].values[0]
        interm_data[f'{drug_id}_SMILES'].append(smiles)
        drug_info_data = drug_info_ws_v2(name)
        if drug_info_data['target'] == "" and name not in not_avail_drug_lst:
            not_avail_drug_lst.append(name)
        interm_data[f'{drug_id}_target'].append(drug_info_data['target'])
        interm_data[f'{drug_id}_target_path'].append(drug_info_data['pathway'])

100%|████████████████████████████████████████████████████████████████████████████████████████| 692/692 [13:28<00:00,  1.17s/it]


In [12]:
interm_df = pd.DataFrame(interm_data)
print(interm_df.shape)
interm_df.head()

(692, 17)


,cell_id,mut_status,drug_1_name,drug_1_SMILES,drug_1_target,drug_1_target_path,drug_1_response,drug_2_name,drug_2_SMILES,drug_2_target,drug_2_target_path,drug_2_response,drug_3_name,drug_3_SMILES,drug_3_target,drug_3_target_path,drug_3_response
0,ACH-000001,"NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, W...",IMATINIB,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,"ABL, KIT, PDGFR","Other, kinases",0.406700,AT7867,C1CNCCC1(C2=CC=C(C=C2)C3=CNN=C3)C4=CC=C(C=C4)Cl,AKT,PI3K/MTOR signaling,0.977759,UNC1215,C1CCN(C1)C2CCN(CC2)C(=O)C3=CC(=C(C=C3)C(=O)N4C...,L3MBTL3,Chromatin other,0.987051
1,ACH-000002,"CDKN2A, CEBPA, KDM5C, NRAS, RPTOR",VX-11E,CC1=CN=C(NC2=C(Cl)C=C(F)C=C2)N=C1C3=CNC(C(N[C@...,ERK2,ERK MAPK signaling,0.402122,SABUTOCLAX,CC1=CC2=C(C(=C(C=C2C(=C1C3=C(C4=CC(=C(C(=C4C=C...,"BCL2, BCL-XL, BFL1, MCL1",Apoptosis regulation,0.890232,XMD8-92,CCOC1=C(C=CC(=C1)N2CCC(CC2)O)NC3=NC=C4C(=N3)N(...,ERK5,ERK MAPK signaling,0.985743
2,ACH-000004,"FGFR3, POLE, TP53, ZNF703",CETUXIMAB,CC1(C(=C)N(C2=CC=CC=C21)CCCCCC(=O)O)C,EGFR,EGFR signaling,0.468953,SALUBRINAL,C1=CC=C(C=C1)C=CC(=O)NC(C(Cl)(Cl)Cl)NC(=S)NC2=...,EIF2A,Other,0.978807,LEFLUNOMIDE,CC1=C(C=NO1)C(=O)NC2=CC=C(C=C2)C(F)(F)F,Pyrimidine synthesis inhibitor,DNA replication,0.987970
3,ACH-000006,"ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1",AVAGACESTAT,C1=CC(=CC=C1S(=O)(=O)N(CC2=C(C=C(C=C2)C3=NOC=N...,"Amyloid beta20, Amyloid beta40",Other,0.411493,DOCETAXEL,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@...,Microtubule stabiliser,Mitosis,0.976822,KRAS (G12C) INHIBITOR-12,C=CC(=O)N1CCN(CC1)C(=O)CNC2=CC(=C(C=C2O)Cl)I,KRAS (G12C),ERK MAPK signaling,0.991612
4,ACH-000007,"ALK, BRAF, KRAS, MET, MUTYH, NOTCH3, RAD54L, R...",AVAGACESTAT,C1=CC(=CC=C1S(=O)(=O)N(CC2=C(C=C(C=C2)C3=NOC=N...,"Amyloid beta20, Amyloid beta40",Other,0.411493,GSK429286A,CC1=C(C(CC(=O)N1)C2=CC=C(C=C2)C(F)(F)F)C(=O)NC...,"ROCK1, ROCK2",Cytoskeleton,0.972790,GW-843682X,COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C(=O)N)OCC4=...,,,0.988587


In [13]:
len(not_avail_drug_lst)

20

In [14]:
### Some drug info is not available on the GDSC platform?
print(not_avail_drug_lst)

['BRYOSTATIN-1', 'DACOMITINIB', 'ZOLEDRONATE', 'EPOTHILONE-B', 'CANERTINIB', 'JQ1-(+)', 'LCL-161', 'ELEPHANTIN', 'NSC-319726', 'SINULARIN', 'VINCRISTINE', 'CARMUSTINE', 'SU-11274', 'TRICHOSTATIN-A', 'SB-525334', 'TORIN-2', 'RU-SKI-43', 'TUBASTATIN-A', 'NINTEDANIB', 'GW-843682X']


In [15]:
interm_df.to_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_interm_data_task2_simple.csv")

In [16]:
### import interm_df to create the prompt_data
interm_df = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_interm_data_task2_simple.csv", index_col = 0)
print(interm_df.shape)
interm_df.head()

(692, 17)


,cell_id,mut_status,drug_1_name,drug_1_SMILES,drug_1_target,drug_1_target_path,drug_1_response,drug_2_name,drug_2_SMILES,drug_2_target,drug_2_target_path,drug_2_response,drug_3_name,drug_3_SMILES,drug_3_target,drug_3_target_path,drug_3_response
0,ACH-000001,"NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, W...",IMATINIB,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,"ABL, KIT, PDGFR","Other, kinases",0.406700,AT7867,C1CNCCC1(C2=CC=C(C=C2)C3=CNN=C3)C4=CC=C(C=C4)Cl,AKT,PI3K/MTOR signaling,0.977759,UNC1215,C1CCN(C1)C2CCN(CC2)C(=O)C3=CC(=C(C=C3)C(=O)N4C...,L3MBTL3,Chromatin other,0.987051
1,ACH-000002,"CDKN2A, CEBPA, KDM5C, NRAS, RPTOR",VX-11E,CC1=CN=C(NC2=C(Cl)C=C(F)C=C2)N=C1C3=CNC(C(N[C@...,ERK2,ERK MAPK signaling,0.402122,SABUTOCLAX,CC1=CC2=C(C(=C(C=C2C(=C1C3=C(C4=CC(=C(C(=C4C=C...,"BCL2, BCL-XL, BFL1, MCL1",Apoptosis regulation,0.890232,XMD8-92,CCOC1=C(C=CC(=C1)N2CCC(CC2)O)NC3=NC=C4C(=N3)N(...,ERK5,ERK MAPK signaling,0.985743
2,ACH-000004,"FGFR3, POLE, TP53, ZNF703",CETUXIMAB,CC1(C(=C)N(C2=CC=CC=C21)CCCCCC(=O)O)C,EGFR,EGFR signaling,0.468953,SALUBRINAL,C1=CC=C(C=C1)C=CC(=O)NC(C(Cl)(Cl)Cl)NC(=S)NC2=...,EIF2A,Other,0.978807,LEFLUNOMIDE,CC1=C(C=NO1)C(=O)NC2=CC=C(C=C2)C(F)(F)F,Pyrimidine synthesis inhibitor,DNA replication,0.987970
3,ACH-000006,"ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1",AVAGACESTAT,C1=CC(=CC=C1S(=O)(=O)N(CC2=C(C=C(C=C2)C3=NOC=N...,"Amyloid beta20, Amyloid beta40",Other,0.411493,DOCETAXEL,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@...,Microtubule stabiliser,Mitosis,0.976822,KRAS (G12C) INHIBITOR-12,C=CC(=O)N1CCN(CC1)C(=O)CNC2=CC(=C(C=C2O)Cl)I,KRAS (G12C),ERK MAPK signaling,0.991612
4,ACH-000007,"ALK, BRAF, KRAS, MET, MUTYH, NOTCH3, RAD54L, R...",AVAGACESTAT,C1=CC(=CC=C1S(=O)(=O)N(CC2=C(C=C(C=C2)C3=NOC=N...,"Amyloid beta20, Amyloid beta40",Other,0.411493,GSK429286A,CC1=C(C(CC(=O)N1)C2=CC=C(C=C2)C(F)(F)F)C(=O)NC...,"ROCK1, ROCK2",Cytoskeleton,0.972790,GW-843682X,COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C(=O)N)OCC4=...,NaN,NaN,0.988587


In [17]:
### Missing drug information about target and target pathway --> set as Unknown
# 'ABT-737'
# 'BRYOSTATIN-1'
# 'QS-11'
# 'JNJ-38877605'
# 'ACETALAX'
# 'NU-7441'
# 'JQ1-(+)'
# 'VINCRISTINE'
# 'SINULARIN'
# 'T-0901317'
# 'SB-505124'
# 'SU-11274'
# 'EPOTHILONE-B'
# 'FILANESIB'

In [18]:
### prompt data
prompt_data = {
    'cell_id': [],
    'prompt': [],
    'answer': []
}

In [19]:
for i in range(len(interm_df)):
    order = list(range(1, N+1)) 
    random.shuffle(order) ### shuffle the order of the drug options
    prompt = "Think step by step and decide the best drug option for the cell line with given mutations: [Drug Name], [Reasoning].\n"
    cell_drug_data = interm_df.iloc[i]
    mut_status = cell_drug_data['mut_status']
    cell_id = cell_drug_data['cell_id']
    idx = 0
    for n in order:
        idx += 1
        drug_name = cell_drug_data[f'drug_{n}_name']
        drug_SMILES = cell_drug_data[f'drug_{n}_SMILES']
        drug_SMILES = drug_SMILES if pd.notna(drug_SMILES) else "Unknown"
        drug_target = cell_drug_data[f'drug_{n}_target']
        drug_target = drug_target if pd.notna(drug_target) else "Unknown"
        drug_target_path = cell_drug_data[f'drug_{n}_target_path']
        drug_target_path = drug_target_path if pd.notna(drug_target_path) else "Unknown"
        prompt += f"Drug {idx}: The drug is {drug_name}. The drug SMILES structure is {drug_SMILES}. Drug target is {drug_target}. Drug target pathway is {drug_target_path}.\n"
    prompt += f"The mutations of the cell line are {mut_status}.\n"
    prompt += "Best drug option: ?"
    prompt_data['cell_id'].append(cell_id)
    prompt_data['prompt'].append(prompt)
    prompt_data['answer'].append(cell_drug_data[f'drug_1_name']) ### best drug option with the lowest AUDRC value

In [20]:
prompt_df = pd.DataFrame(prompt_data)
for i in range(3):
    print(prompt_df.prompt[i])
    print(prompt_df.answer[i])
    print()

Think step by step and decide the best drug option for the cell line with given mutations: [Drug Name], [Reasoning].
Drug 1: The drug is UNC1215. The drug SMILES structure is C1CCN(C1)C2CCN(CC2)C(=O)C3=CC(=C(C=C3)C(=O)N4CCC(CC4)N5CCCC5)NC6=CC=CC=C6. Drug target is L3MBTL3. Drug target pathway is Chromatin other.
Drug 2: The drug is IMATINIB. The drug SMILES structure is CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C)NC4=NC=CC(=N4)C5=CN=CC=C5. Drug target is ABL, KIT, PDGFR. Drug target pathway is Other, kinases.
Drug 3: The drug is AT7867. The drug SMILES structure is C1CNCCC1(C2=CC=C(C=C2)C3=CNN=C3)C4=CC=C(C=C4)Cl. Drug target is AKT. Drug target pathway is PI3K/MTOR signaling.
The mutations of the cell line are NOTCH1, NOTCH3, PIK3R1, PPP2R1A, TP53, TSC2, WHSC1L1.
Best drug option: ?
IMATINIB

Think step by step and decide the best drug option for the cell line with given mutations: [Drug Name], [Reasoning].
Drug 1: The drug is VX-11E. The drug SMILES structure is CC1=CN=C(NC2=C(Cl)C

In [21]:
### train_test_split
### train data
f = open("/home/yingfei/train_celllines_v1_1111.txt", "r")
train_cell_line = [line.strip() for line in f.readlines()]
print(len(train_cell_line))

### test data
f = open("/home/yingfei/test_celllines_v1_1111.txt", "r")
test_cell_line = [line.strip() for line in f.readlines()]
print(len(test_cell_line))

### train_prompt_data
train_prompt_data = prompt_df.loc[prompt_df.cell_id.isin(train_cell_line)].reset_index(drop = True)
print(train_prompt_data.shape)

### test_prompt_data
test_prompt_data = prompt_df.loc[prompt_df.cell_id.isin(test_cell_line)].reset_index(drop = True)
print(test_prompt_data.shape)

623
69
(623, 3)
(69, 3)


In [22]:
train_prompt_data.to_csv("/data/yingfei/cancer_data/llm_prompt_data/train_prompt_data_task2_simple.csv", index = False)
test_prompt_data.to_csv("/data/yingfei/cancer_data/llm_prompt_data/test_prompt_data_task2_simple.csv", index = False)